In [8]:
#Please comment your code clearly, as the clarity of your code will also
# be evaluated. 

# 4.1


 # unlike in YG's example, no $\Pi$

In [ ]:
def predict_individual_shares(delta, X, Gamma, v_draws, by_market):
    """
    Returns a dictionary keyed by market_id,
    where each value is a 2D array of shape (num_products_in_mkt, n_draws)
    containing the shares s_{i,j,t} for each product j and draw i.
    
    Parameters
    ----------
    delta : array of shape (N,)
        The mean-utility for each product in the full dataset (already solved via contraction).
    X : array (N x 2)
        The observables [p, x] for each product.
    Gamma : 2x2 array
        Current random-coeff parameter matrix.
    v_draws : (n_draws x 2)
        The taste‐shock draws for [price, x].
    by_market : array of shape (N,)
        Market IDs labeling each row of (delta, X).
    """
    n_draws = v_draws.shape[0]
    # random coefficients (n_draws x 2)
    rc = (Gamma @ v_draws.T).T
    
    unique_mkts = np.unique(by_market)
    s_by_market = {}

    for m_id in unique_mkts:
        idx = (by_market == m_id)
        delta_m = delta[idx]               # (#products_in_market,)
        X_m     = X[idx, :]               # (#products_in_market, 2)
        
        # utilities by draw => (num_products_in_mkt, n_draws)
        mu = X_m @ rc.T
        util = delta_m[:, None] + mu
        
        exp_util = np.exp(util)
        denom    = exp_util.sum(axis=0)   # shape (n_draws,)
        s_ij     = exp_util / denom       # shape (#products_in_mkt, n_draws)
        
        s_by_market[m_id] = s_ij
    
    return s_by_market
import numpy as np

def compute_elasticities_for_one_market(s_ij, prices, shares):
    """
    Compute the JxJ elasticity matrix for a single market.
    
    Parameters
    ----------
    s_ij : 2D array (J x n_draws)
        The *individual-draw* shares for each of the J products (excluding outside good if desired).
    prices : 1D array (J,)
        Observed prices p_j,t for each of the J products in this market.
    shares : 1D array (J,)
        Observed shares s_j,t for each product (the average across i, or from your dataframe).
        
    Returns
    -------
    E : 2D array (J x J)
        The elasticity matrix.
    """
    J, n_draws = s_ij.shape
    E = np.zeros((J, J))

    # For convenience, compute the average across draws for each product:
    # But typically you already have "shares" from the data or from predict_shares(...).
    # Here we just confirm that shares ~ np.mean(s_ij, axis=1).
    # We'll use the given 'shares' argument, but you could check consistency.
    
    for j in range(J):
        for k in range(J):
            # derivative wrt p_k:
            if j == k:
                # own derivative: mean_i [ s_ij * (1 - s_ij) ]
                deriv = np.mean(s_ij[j,:] * (1.0 - s_ij[j,:]))
                # elasticity = - (p_j / s_j) * deriv
                E[j,k] = - (prices[j] / shares[j]) * deriv
            else:
                # cross derivative: mean_i [ s_ij * s_ik ]
                deriv = np.mean(s_ij[j,:] * s_ij[k,:])
                # elasticity = (p_k / s_j) * deriv
                E[j,k] = (prices[k] / shares[j]) * deriv
    return E
def compute_average_elasticities(
    df_all,        # your full dataframe with an outside good included
    delta, X, Gamma,  # final parameter estimates
    v_draws, by_market
):
    """
    For each market t, compute the JxJ elasticity matrix, then
    take the simple average across all t (elementwise).
    
    Returns
    -------
    E_avg : 2D array (J x J)
        The average elasticity matrix across markets.
    """
    # 1) Get s_{i,j,t} for each market from the final delta
    s_dict = predict_individual_shares(delta, X, Gamma, v_draws, by_market)

    # We'll store elasticity matrices for each market in a list
    E_list = []
    
    unique_mkts = df_all["market"].unique()
    
    for m_id in unique_mkts:
        # subset the dataframe to market m_id
        df_m = df_all[df_all["market"] == m_id].copy()
        # drop the outside good if "choice=0" so we get just inside goods
        df_m = df_m[df_m["choice"] > 0]
        df_m.sort_values("choice", inplace=True)
        
        # prices and shares (observed) for the inside goods
        p_m = df_m["p"].values
        s_m = df_m["shares"].values
        
        # s_ij for this market (the draws-based matrix)
        s_ij = s_dict[m_id]
        
        # also drop row 0 if that corresponds to the outside good:
        # (depends on how you stored them, typically row 0 is outside good)
        # you can find the index by  (df_m["choice"]==0) etc. 
        # but if you sorted by choice, and outside good is choice=0, 
        # that row might be at the front. So let's just do the same filtering:
        # We want s_ij only for inside products:
        outside_mask = (df_all["market"] == m_id) & (df_all["choice"] == 0)
        n_outside = outside_mask.sum()
        # so skip the first 'n_outside' rows if indeed outside good was at the top
        s_ij_inside = s_ij[n_outside:,:]  
        
        # Now compute the JxJ for this market
        E_m = compute_elasticities_for_one_market(s_ij_inside, p_m, s_m)
        E_list.append(E_m)
    
    # Convert to 3D array to average
    # (assuming all markets have the same # of inside products J)
    E_stack = np.stack(E_list, axis=2)  # shape (J, J, T)
    E_avg = np.mean(E_stack, axis=2)    # shape (J, J)
    
    return E_avg


In [77]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import statsmodels.api as sm
import statsmodels.sandbox.regression.gmm as gmm
from statsmodels.sandbox.regression.gmm import IV2SLS

##############################################################################
# 1. Read Data & Construct Outside Good
##############################################################################
def read_data_with_outside_good(csv_path):
    """
    Reads a dataset that has columns:
        market, choice, shares, p, x, z1, z2, ...
    If the outside good (choice=0) is *not* in the CSV, this function
    creates a synthetic row for each market with p0=x0=0, share=1-sum(shares_in_market),
    and zeros for instruments if needed.
    """
    print(">>> Reading CSV and ensuring each market has an outside good row...")
    df = pd.read_csv(csv_path).copy()
    df.sort_values(by=["market", "choice"], inplace=True)
    df.reset_index(drop=True, inplace=True)

    frames = []
    for m in df["market"].unique():
        this_mkt = df.loc[df["market"] == m].copy()
        outside_row = this_mkt.loc[this_mkt["choice"] == 0]

        if outside_row.empty:
            sum_inside = this_mkt["shares"].sum()
            outside_share = max(0.0, 1.0 - sum_inside)
            row_outside = {
                "market": m,
                "choice": 0,
                "shares": outside_share,
                "p": 0.0,
                "x": 0.0,
            }
            # For z1,z2,... set to 0
            for c in this_mkt.columns:
                if c.startswith("z"):
                    row_outside[c] = 0.0
            this_mkt = pd.concat([this_mkt, pd.DataFrame([row_outside])],
                                 ignore_index=True)

        this_mkt.sort_values("choice", inplace=True)
        frames.append(this_mkt)

    df_out = pd.concat(frames, ignore_index=True)
    df_out.sort_values(by=["market", "choice"], inplace=True)
    df_out.reset_index(drop=True, inplace=True)
    return df_out


##############################################################################
# 2. Draw (vi) ~ N(0, I)
##############################################################################
def draw_taste_shocks(n_draws=50, seed=0):
    """
    Return an array of shape (n_draws, 2) of normal(0,1) draws
    for random coefficients dimension=2 (e.g., for price & x).
    """
    rng = np.random.default_rng(seed)
    v = rng.normal(0.0, 1.0, size=(n_draws, 2))
    return v


##############################################################################
# 3. Predict Shares (Vectorized)
##############################################################################
def predict_shares(delta, X, Gamma, v_draws, by_market):
    """
    Vectorized share prediction:
      s_j = (1/n_draws) sum_{i=1..n_draws} 
             exp( delta_j + X_j @ (Gamma v_i) )
           / sum_{k} exp( delta_k + X_k @ (Gamma v_i) ),
    by market.
    """
    n_draws = v_draws.shape[0]
    s_pred = np.zeros_like(delta)

    # Precompute random coefficients for all draws => shape (n_draws, 2)
    rc = (Gamma @ v_draws.T).T  # shape (n_draws, 2)

    unique_mkts = np.unique(by_market)
    for m_id in unique_mkts:
        idx = (by_market == m_id)

        delta_m = delta[idx]          # (#products_in_market,)
        X_m     = X[idx,:]            # (#products_in_market, 2)

        mu = X_m @ rc.T                             # (#products_in_market, n_draws)
        util = delta_m[:, None] + mu                # (#products_in_market, n_draws)
        exp_util = np.exp(util)

        denom = exp_util.sum(axis=0)                # shape (n_draws,)
        s_pred[idx] = (exp_util / denom).mean(axis=1)

    return s_pred


##############################################################################
# 4. Contraction Mapping
##############################################################################
def contraction_mapping(s_obs, X, Gamma, v_draws, by_market,
                       tol=1e-6, max_iter=100000, print_every=500):
    """
    Solves for delta_j subject to s_pred(delta) = s_obs.
    Outside good => delta_0=0. We only update inside goods in the iteration.
    """
    print(">>> Starting contraction mapping to solve for delta_j...")
    outside_idx = np.all(np.isclose(X, 0.0), axis=1)  # index of outside good
    inside_idx  = ~outside_idx

    s_obs_safe = s_obs.clip(1e-16)
    # Initialize delta via log(s_j)
    delta = np.log(s_obs_safe)
    delta[outside_idx] = 0.0  # delta for outside good set to 0

    for iteration in range(max_iter):
        if (iteration % print_every == 0) and (iteration > 0):
            pct = 100.0 * iteration / max_iter
            print(f"    Contraction iteration: {iteration}/{max_iter} ({pct:.1f}% done)")

        s_pred = predict_shares(delta, X, Gamma, v_draws, by_market)
        s_pred_safe = s_pred.clip(1e-16)

        delta_new = delta.copy()
        delta_new[inside_idx] = (
            delta[inside_idx]
            + np.log(s_obs_safe[inside_idx])
            - np.log(s_pred_safe[inside_idx])
        )
        delta_new[outside_idx] = 0.0

        sup_norm = np.max(np.abs(delta_new - delta))
        delta = delta_new
        if sup_norm < tol:
            print(f">>> Contraction mapping converged after {iteration+1} iterations.")
            break

    return delta


##############################################################################
# 5a. GMM Objective
##############################################################################
def gmm_objective(params, s_obs, X, Z, v_draws, by_market, W):
    """
    Objective = (Z' xi)' W (Z' xi),
    where xi = delta - X beta and delta solves the contraction mapping.
    """
    gamma11, gamma21, gamma22 = params
    Gamma = np.array([
        [gamma11, 0.0],
        [gamma21, gamma22]
    ])

    # Solve for delta
    delta = contraction_mapping(s_obs, X, Gamma, v_draws, by_market,
                                print_every=9999999)

    # 2SLS to get beta
    Z_with_const = sm.add_constant(Z, has_constant='add')
    iv_model = IV2SLS(endog=delta, exog=X, instrument=Z_with_const)
    iv_results = iv_model.fit()
    beta = iv_results.params

    xi = delta - X @ beta
    m = Z.T @ xi
    obj = m.T @ W @ m

    return obj


##############################################################################
# 5b. Helpers for gradient-based GMM
##############################################################################
def approximate_delta_grad(gamma, s_obs, X, v_draws, by_market,
                           h=1e-5):
    """
    Numerically approximate d(delta)/dGamma at the current Gamma.
    gamma = [gamma11, gamma21, gamma22]
    Returns: ddelta_dgamma, shape (N, 3)
    """
    gamma11, gamma21, gamma22 = gamma
    Gamma_base = np.array([[gamma11, 0.0],
                           [gamma21, gamma22]])
    delta_base = contraction_mapping(s_obs, X, Gamma_base, v_draws, by_market,
                                     print_every=9999999)

    n = len(delta_base)
    ddelta_dgamma = np.zeros((n, 3))

    for j in range(3):
        step = np.zeros(3)
        step[j] = h

        Gamma_pert = np.array([
            [gamma11 + step[0], 0.0],
            [gamma21 + step[1], gamma22 + step[2]]
        ])
        delta_pert = contraction_mapping(s_obs, X, Gamma_pert, v_draws, by_market,
                                         print_every=9999999)
        ddelta_dgamma[:, j] = (delta_pert - delta_base) / (step[j])

    return ddelta_dgamma

def compute_2sls_beta(delta, X, Z):
    """
    Compute 2SLS coefficient vector from model: delta = X beta + error
    with instruments Z (including a constant).
    """
    iv_model = gmm.IV2SLS(endog=delta, exog=X, instrument=Z)
    results = iv_model.fit()
    return results.params

def gmm_objective_and_grad(gamma, s_obs, X, Z, v_draws, by_market, W):
    """
    Returns: (objective, gradient wrt gamma),
    where gamma = [gamma11, gamma21, gamma22].
    """
    # 1) Build Gamma, solve for delta
    gamma11, gamma21, gamma22 = gamma
    Gamma = np.array([[gamma11, 0.0],
                      [gamma21, gamma22]])
    delta = contraction_mapping(s_obs, X, Gamma, v_draws, by_market,
                                print_every=9999999)

    # 2) Add constant in instruments
    ones = np.ones((Z.shape[0], 1))
    Z_with_const = np.hstack([ones, Z])

    # 3) Compute 2SLS beta
    beta_2sls = compute_2sls_beta(delta, X, Z_with_const)

    # 4) xi = delta - X beta
    xi = delta - X @ beta_2sls

    # 5) Evaluate objective
    m = Z.T @ xi
    obj = m.T @ W @ m

    # 6) We need d(delta)/d(gamma)
    ddelta_dgamma = approximate_delta_grad(gamma, s_obs, X, v_draws, by_market)

    # 7) Then d(beta)/d(gamma)
    ZtZ_inv = np.linalg.inv(Z_with_const.T @ Z_with_const)
    PZc = Z_with_const @ ZtZ_inv @ Z_with_const.T
    XPZX = X.T @ PZc @ X
    XPZX_inv = np.linalg.inv(XPZX)
    B = XPZX_inv @ (X.T @ PZc)
    n = len(delta)
    M = np.eye(n) - X @ B  # shape (N, N)

    # 8) derivative wrt xi of the objective: g_xi = 2 Z W (Z' xi)
    g_xi = 2.0 * (Z @ (W @ (Z.T @ xi)))  # shape (N,)

    # 9) gradient wrt gamma
    ddelta = M @ ddelta_dgamma  # shape (N,3)
    grad_gamma = ddelta.T @ g_xi  # shape (3,)
    print("beta")
    print(beta_2sls)
    print("obj")
    print(obj)
    return obj, grad_gamma

##############################################################################
# 5c. Utility: get current xi for a given Gamma (for building W2)
##############################################################################
def get_xi_for_gamma(gamma, s_obs, X, Z, v_draws, by_market):
    """
    Returns xi (N-vector), plus the final delta and the 2SLS beta,
    all given the current gamma = [gamma11, gamma21, gamma22].
    """
    gamma11, gamma21, gamma22 = gamma
    Gamma = np.array([[gamma11, 0.0],
                      [gamma21, gamma22]])
    # Solve for delta
    delta = contraction_mapping(s_obs, X, Gamma, v_draws, by_market,
                                print_every=9999999)

    # 2SLS
    Z_with_const = sm.add_constant(Z, has_constant='add')
    iv_model = IV2SLS(endog=delta, exog=X, instrument=Z_with_const)
    iv_results = iv_model.fit()
    beta = iv_results.params

    xi = delta - X @ beta
    return xi, delta, beta


##############################################################################
# 6. Two-Step GMM with Progress Messages (NEW)
##############################################################################
def estimate_blp_iterative_gmm(df, n_draws=50, method="BFGS", use_analytic_grad=True):
    """
    1) Build Z, initial W1 = (Z'Z)^(-1)
    2) Minimize GMM => res1
    3) Recompute W => W2 from res1's estimates
    4) Minimize => res2
    Returns both passes.
    """
    df.sort_values(by=["market","choice"], inplace=True)
    df.reset_index(drop=True, inplace=True)

    s_obs = df["shares"].values
    X = df[["p","x"]].values
    z_cols = [c for c in df.columns if c.startswith("z")]
    if not z_cols:
        raise ValueError("No instrument columns found!")
    Z = df[z_cols].values

    by_market = df["market"].values

    print(">>> Building first-stage weighting matrix W1 = (Z'Z)^-1 ...")
    ZZ = Z.T @ Z
    eye_k = np.eye(ZZ.shape[0])
    W1 = np.linalg.inv(ZZ + 1e-12 * eye_k)

    # Draw taste shocks
    v_draws = draw_taste_shocks(n_draws=n_draws, seed=0)

    init_params = np.array([1,1,1])
    print(">>> Starting FIRST pass GMM optimization...")
    #standard devs must be positive. enforced by these bounds.
    bounds = [(0, None) if i in [0, 2] else (None, None) 
             for i in range(len(init_params))]   
    # Helper for first pass
    if use_analytic_grad:
        def obj_grad_fun_1(par):
            return gmm_objective_and_grad(
                par, s_obs, X, Z, v_draws, by_market, W1
            )
        res1 = minimize(
            obj_grad_fun_1,
            init_params,
            method=method,
            jac=True,
            bounds = bounds,
            options={"disp": True, "maxiter": 40}
        )
    else:
        def obj_fun_1(par):
            return gmm_objective(
                par, s_obs, X, Z, v_draws, by_market, W1
            )
        res1 = minimize(
            obj_fun_1,
            init_params,
            method=method,
            bounds = bounds,
            options={"disp": True, "maxiter": 40}
        )

    print(f"    [FIRST PASS] success={res1.success}, "
          f"params={res1.x}, obj={res1.fun:.6g}")
    #Final values for computing elasticities
    Gamma_hat = np.array([[res1.x[0], 0],
                  [res1.x[1],res1.x[2]]])
    delta_hat = contraction_mapping(s_obs, X, Gamma_hat, v_draws, by_market,
                                print_every=9999999)
    E_avg = compute_average_elasticities(
    df_all, 
    delta_hat, 
    X=df_all[["p","x"]].values, 
    Gamma=Gamma_hat, 
    v_draws=v_draws, 
    by_market=df_all["market"].values
)
    # ------------------------
    # 2nd Step: Build W2 using \hat{x}\hat{x}' from first pass
    # ------------------------
    # print("\n>>> Building second-stage weighting matrix W2 ...")
    # gamma_hat_1 = res1.x
    # xi_1, _, _ = get_xi_for_gamma(gamma_hat_1, s_obs, X, Z, v_draws, by_market)
    # # moment = Z_i * xi_i.  We'll form S = sum_i (Z_i xi_i)(Z_i xi_i)' => (Z' * diag(xi) * Z),
    # # typically with a 1/N factor as well. We'll do the simplest approach:

    # # shape: (N, K)
    # ZX = Z * xi_1.reshape(-1,1)  # elementwise multiply each column of Z by xi
    # S_hat = ZX.T @ ZX  # shape (K,K), sum of outer products
    # # Optionally scale by 1/N or 1/(N-K).
    # N = Z.shape[0]
    # S_hat /= N

    # # W2 is inverse of S_hat
    # # (Add small ridge if needed for invertibility)
    # ridge = 1e-12 * np.eye(S_hat.shape[0])
    # W2 = np.linalg.inv(S_hat + ridge)

    # # 2nd pass optimization
    # print(">>> Starting SECOND pass GMM optimization using W2...")
    # if use_analytic_grad:
    #     def obj_grad_fun_2(par):
    #         return gmm_objective_and_grad(
    #             par, s_obs, X, Z, v_draws, by_market, W2
    #         )
    #     res2 = minimize(
    #         obj_grad_fun_2,
    #         res1.x,   #  first pass estimate as initial
    #         method=method,
    #         jac=True,
    #         options={"disp": True, "maxiter": 10}
    #     )
    # else:
    #     def obj_fun_2(par):
    #         return gmm_objective(
    #             par, s_obs, X, Z, v_draws, by_market, W2
    #         )
    #     res2 = minimize(
    #         obj_fun_2,
    #         res1.x,
    #         method=method,
    #         options={"disp": True, "maxiter": 10}
    #     )

    # print(f"    [SECOND PASS] success={res2.success}, "
    #       f"params={res2.x}, obj={res2.fun:.6g}")

    return res1,E_avg#, res2


##############################################################################
# 7. Main 
##############################################################################
if __name__ == "__main__":
    # 1) Read data, ensuring outside good rows
    df_all = read_data_with_outside_good("ps1_ex4.csv")

    # 2)  Perform the two-step GMM estimation
    first_pass,E_avg = estimate_blp_iterative_gmm(
        df_all,
        n_draws=50,
        method='l-bfgs-b' ,
        use_analytic_grad=True
    )

    print("\n=== FINAL RESULTS ===")
    print("[FIRST PASS] success=", first_pass.success)
    print("  params=", first_pass.x)
    print("  objective=", first_pass.fun)
    #print("[SECOND PASS] success=", second_pass.success)
    #print("  params=", second_pass.x)
    #print("  objective=", second_pass.fun)


>>> Reading CSV and ensuring each market has an outside good row...
>>> Building first-stage weighting matrix W1 = (Z'Z)^-1 ...
>>> Starting FIRST pass GMM optimization...
>>> Starting contraction mapping to solve for delta_j...
>>> Contraction mapping converged after 310 iterations.
>>> Starting contraction mapping to solve for delta_j...
>>> Contraction mapping converged after 310 iterations.
>>> Starting contraction mapping to solve for delta_j...
>>> Contraction mapping converged after 310 iterations.
>>> Starting contraction mapping to solve for delta_j...
>>> Contraction mapping converged after 310 iterations.
>>> Starting contraction mapping to solve for delta_j...
>>> Contraction mapping converged after 310 iterations.
beta
[-0.40535423 -2.41233908]
obj
881.4338015082142
>>> Starting contraction mapping to solve for delta_j...
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly 

NameError: name 'compute_average_elasticities' is not defined

In [75]:
Gamma = np.array([[first_pass.x[0], 0.00000000e+00],
                  [first_pass.x[1],first_pass.x[2]]])
Gamma


array([[ 4.65683615e+00,  0.00000000e+00],
       [-1.22243271e-01,  1.40879983e-03]])

In [76]:
# Multiply Gamma by its transpose
Omega = np.dot(Gamma, Gamma.T)
Omega

array([[ 2.16861230e+01, -5.69266886e-01],
       [-5.69266886e-01,  1.49454021e-02]])

#4.2    
 For each market, compute cross and own product elasticities. Average your results across
markets and present them in a J ×J table whose (i, j) element contains the (average) elasticity
of product i with respect to an increase in the price of product j. What’s the main difference
when compared with the table of elasticities you found in 2.2?


see slide 14 of Yonggeun's slides. we need to use the prices, shares, alpha_i (not clear how that maps to here) , and density of q_i (not sure how that maps to here), and density of v_i (assumed above)

# 4.3 look at average price and share for products, across market, using $\Gamma$, what is driving differences in prices and market shares

df_all

In [11]:
import pandas as pd

# 1. Group by product, compute mean for p, x, and shares
table = (
    df_all
    .groupby('choice', as_index=False)
    .agg(
        avg_price=('p','mean'),
        avg_x=('x','mean'),
        avg_share=('shares','mean')
    )
)

# 2. Convert to LaTeX
latex_table = table.to_latex(
    index=False,
    float_format="%.4f",    # set decimal precision as needed
    caption="Average Price, x, and Share by Product",
    label="tab:avg_price_x_share"
)
print(latex_table)


\begin{table}
\caption{Average Price, x, and Share by Product}
\label{tab:avg_price_x_share}
\begin{tabular}{rrrr}
\toprule
choice & avg_price & avg_x & avg_share \\
\midrule
0 & 0.0000 & 0.0000 & 0.3848 \\
1 & 0.0024 & -0.0193 & 0.0988 \\
2 & 0.0023 & -0.0260 & 0.0891 \\
3 & 2.0191 & -0.0813 & 0.0430 \\
4 & 1.7516 & -0.1801 & 0.0393 \\
5 & 3.5770 & 1.6926 & 0.1517 \\
6 & 4.4429 & 2.0024 & 0.1932 \\
\bottomrule
\end{tabular}
\end{table}



# 4.4 PyBLP

In [12]:
%pip install pyblp
import pyblp


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [46]:
df_blp = df_all.copy()
df_blp = df_blp.rename(columns={'p': 'prices'})
df_blp = df_blp.rename(columns={'choice': 'product_ids'})
df_blp = df_blp.rename(columns={'market': 'market_ids'})



df_blp = df_blp.rename(columns={'z1': 'demand_instruments0'})
df_blp = df_blp.rename(columns={'z2': 'demand_instruments1'})
df_blp = df_blp.rename(columns={'z3': 'demand_instruments2'})
df_blp = df_blp.rename(columns={'z4': 'demand_instruments3'})
df_blp = df_blp.rename(columns={'z5': 'demand_instruments4'})
df_blp = df_blp.rename(columns={'z6': 'demand_instruments5'})

#drop outside good.
df_blp = df_blp[df_blp['product_ids'] != 0]

# X1 are things that have homog effects
#X2 are (potentially same things) with heterog. effects
X1_formulation = pyblp.Formulation('0+ prices + x') # no constant term
X2_formulation = pyblp.Formulation('0 + prices + x') # random coefficient for outside good is not a thing in this model.
product_formulations = (X1_formulation,X2_formulation)
product_formulations



(prices + x, prices + x)

In [47]:
df_blp

,market_ids,product_ids,shares,prices,x,demand_instruments0,demand_instruments1,demand_instruments2,demand_instruments3,demand_instruments4,demand_instruments5
1,1,1,0.0099,0.000278,-0.210973,-6.363361,-5.509449,-5.371022,-1.583637,-1.937069,-0.341810
2,1,2,0.2085,0.000457,-1.557577,-5.198161,-4.396036,-4.265266,-2.862346,-3.498094,-2.330125
3,1,3,0.0090,1.016838,-0.799984,-0.040836,0.835758,-0.520122,-0.797271,0.768238,-0.973296
4,1,4,0.0348,0.109472,-1.338366,-1.538905,-1.172700,0.189448,-1.798173,1.519424,-0.264684
5,1,5,0.3661,5.387786,3.316330,0.758555,-0.700470,0.723864,-0.370940,-1.288456,0.080449
...,...,...,...,...,...,...,...,...,...,...,...
695,100,2,0.0900,0.001302,-0.240857,-5.123659,-4.521685,-5.946617,-2.672067,-2.441713,0.872459
696,100,3,0.0008,2.987678,-0.172630,1.530545,1.190138,-0.999596,-1.673909,0.313007,1.037242
697,100,4,0.0007,3.082019,-0.061763,1.688702,-0.331869,0.250214,0.699356,0.445912,-0.737721
698,100,5,0.0364,1.826103,1.276445,0.121505,0.547611,-1.490001,-0.141559,-0.220383,0.643245


In [48]:
df_blp.product_ids.unique()

array([1, 2, 3, 4, 5, 6])

In [49]:
mc_integration = pyblp.Integration('monte_carlo', size=50, specification_options={'seed': 0})
mc_integration



Configured to construct nodes and weights with Monte Carlo simulation with options {seed: 0}.

In [50]:
mc_problem = pyblp.Problem(product_formulations, df_blp, integration=mc_integration)
mc_problem

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N    I     K1    K2    MD 
---  ---  ----  ----  ----  ----
100  600  5000   2     2     7  

Formulations:
       Column Indices:           0      1 
-----------------------------  ------  ---
 X1: Linear Characteristics    prices   x 
X2: Nonlinear Characteristics  prices   x 


Dimensions:
 T    N    I     K1    K2    MD 
---  ---  ----  ----  ----  ----
100  600  5000   2     2     7  

Formulations:
       Column Indices:           0      1 
-----------------------------  ------  ---
 X1: Linear Characteristics    prices   x 
X2: Nonlinear Characteristics  prices   x 

In [61]:
lbfgsb = pyblp.Optimization(
    'l-bfgs-b',
    {
        'gtol': 1e-7,
      
    }
)

none_array = np.full((2, 2), None)
zero_array = np.array([[0, None], [None, 0]]) # just diagonal has to be positive

In [62]:
# need to restrict parameters to be lower triangular. I think. Unclear. There's some documentation here talking about Cholesky decompositions.
# Does seem like they ahve the taste shocks baked in here to some degree 'It is common to assume that 𝑓(𝛽𝑖∣𝜃) follows a multivariate normal distribution, and to break it up into three parts: 

# Might need to do something for beta too. 
results = mc_problem.solve(sigma= np.ones((2,2)), optimization=lbfgsb,
                           sigma_bounds = (zero_array,none_array))
results

Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:     prices            x        |  Sigma Squared:     prices            x      
------  -------------  -------------  |  --------------  -------------  -------------
prices  +1.000000E+00                 |      prices      +1.000000E+00  +1.000000E+00
  x     +1.000000E+00  +1.000000E+00  |        x         +1.000000E+00  +2.000000E+00

Nonlinear Coefficient Lower Bounds:
Sigma:     prices            x      
------  -------------  -------------
prices  +0.000000E+00               
  x         -INF       +0.000000E+00

Nonlinear Coefficient Upper Bounds:
Sigma:     prices            x      
------  -------------  -------------
prices      +INF                    
  x         +INF           +INF     

Starting optimization ...

GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Projected                                               
Step     Time       Itera

Problem Results Summary:
GMM     Objective      Projected    Reduced Hessian  Reduced Hessian  Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Gradient Norm  Min Eigenvalue   Max Eigenvalue   Shares   Condition Number  Condition Number 
----  -------------  -------------  ---------------  ---------------  -------  ----------------  -----------------
 2    +1.614963E+02  +4.551126E-06   +5.220717E+01    +1.588824E+02      0      +1.724003E+01      +7.667095E+06  

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:00:03       Yes          18           30          22615        70262   

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
Sigma:      prices              x         |  Sigma Squared:      prices              x       
------  ---------------  ------

In [20]:
np.ones

<function numpy.ones(shape, dtype=None, order='C', *, device=None, like=None)>